## Índice

1. [General Statistics](#1)
2. [Boxplots](#2)
3. [First Integer Helicity Betta vs Gamma](#3)
4. [Sum Absolute Helicity Betta vs Gamma](#4)
5. [First Integer Enstrophy Betta vs Gamma](#5)
6. [3D Curl Comparisons](#6)
7. [Individual 3D Curl Baseline vs Experimental Color](#7)
8. [Velocity Field Baseline Nivel](#8)
9. [Velocity Field MEI Nivel](#9)
10. [Comparison Moduli before and after for MEI](#10)
11. [Comparison Derivatives Histograms](#11)
12. [Comparison Time Series Laser](#12)


In [40]:
root_folder = r"F:\Datos\20Sep24"

triads = [
        (0, 2, 4),  # T1 (FRONTAL, DERECHA, SUPERIOR)
        (1, 3, 5),  # T2 (TRASERO, IZQUIERDA, INFERIOR)
        (0, 3, 4),  # T3 (FRONTAL, IZQUIERDA, SUPERIOR)
        (1, 2, 5),  # T4 (TRASERO, DERECHA, INFERIOR)
        (0, 2, 5),  # T5 (FRONTAL, DERECHA, INFERIOR)
        (1, 3, 4),  # T6 (TRASERO, IZQUIERDA, SUPERIOR)
        (0, 3, 5),  # T7 (FRONTAL, IZQUIERDA, INFERIOR)
        (1, 2, 4),  # T8 (TRASERO, DERECHA, SUPERIOR)
        (2, 4, 5),  # T9 (DERECHA, SUPERIOR, INFERIOR)
        (0, 1, 3),  # T10 (IZQUIERDA, FRONTAL, TRASERO)
        (3, 4, 5),  # T11 (IZQUIERDA, SUPERIOR, INFERIOR)
        (0, 1, 2)   # T12 (DERECHA, FRONTAL, TRASERO)
]

# Horarios ajustados de mediciones de línea base y de intervenciones experimentales

lab_b_matutino_base_times = {
    '01': "10:39:59.065",
    '02': "11:19:56.844",
    '03': "12:00:00.896",
    '04': "12:39:59.139",
    '05': "13:19:59.461"
}

# Horarios de intervenciones experimentales
lab_b_matutino_intervention_times = {
    '01': "10:59:59.414",
    '02': "11:39:59.104",
    '03': "12:19:59.778",
    '04': "13:00:01.295"
}

# Horarios de mediciones de línea base para Lab_A en turno matutino
lab_a_matutino_base_times = {
    '01': "10:39:59.065",
    '02': "11:19:56.844",
    '03': "12:00:00.896",
    '04': "12:39:59.139",
    '05': "13:19:59.461"
}

# Horarios de intervenciones experimentales para Lab_A en turno matutino
lab_a_matutino_intervention_times = {
    '01': "10:59:59.414",
    '02': "11:39:59.104",
    '03': "12:19:59.778",
    '04': "13:00:01.295"
}

n_change = {"Lab_Betta":"Lab_Betta", "Lab_Gamma":"Lab_Gamma"}

triad_names = {
    "FRT": "Frontal-Derecho-Superior",  
    "PLB": "Trasero-Izquierdo-Inferior",  
    "FLT": "Frontal-Izquierdo-Superior",  
    "PRB": "Trasero-Derecho-Inferior",  
    "FRB": "Frontal-Derecho-Inferior",  
    "PLT": "Trasero-Izquierdo-Superior",  
    "FLB": "Frontal-Izquierdo-Inferior",  
    "PRT": "Trasero-Derecho-Superior",  
    "RTB": "Derecho-Superior-Inferior",  
    "FLP": "Frontal-Izquierdo-Trasero",  
    "LTB": "Izquierdo-Superior-Inferior",  
    "FRP": "Frontal-Derecho-Trasero"
}



## 1

$$
\Huge \text{General Statistcs}
$$

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew, iqr
from matplotlib.backends.backend_pdf import PdfPages
import openpyxl
from datetime import datetime

# Función para leer datos del sensor desde un archivo
def read_sensor_data(filepath):
    data = []
    timestamps = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except UnicodeDecodeError:
        print(f"Unicode decode error in file {filepath}")
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return timestamps, np.array(data).T

# Función para analizar el contenido de una carpeta
def analyze_folder(root_folder):
    data_per_file = {}
    for root, dirs, files in os.walk(root_folder):
        # Evitar carpetas llamadas "Data Analysis"
        if 'Data Analysis' in dirs:
            dirs.remove('Data Analysis')

        for filename in files:
            if filename.startswith("0") and filename.endswith(("negra.txt")):
                file_path = os.path.join(root, filename)
                timestamps, data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = (timestamps, data, filename)
    return data_per_file

# Función para generar estadísticas de los datos
def generate_statistics(data):
    return {
        'mean': np.mean(data, axis=1),
        'median': np.median(data, axis=1),
        'variance': np.var(data, axis=1),
        'stdDev': np.std(data, axis=1),
        'min': np.min(data, axis=1),
        'max': np.max(data, axis=1),
        'firstQuartile': np.percentile(data, 25, axis=1),
        'thirdQuartile': np.percentile(data, 75, axis=1),
        'iqr': iqr(data, axis=1),
        'trimmedVariance': [np.var(np.trim_zeros(np.sort(sensor_data))) for sensor_data in data],
        'kurtosis': kurtosis(data, axis=1, fisher=True, bias=False),
        'skewness': skew(data, axis=1, bias=False)
    }

# Función para extraer la fecha del directorio root_folder
def extract_date_from_root_folder(root_folder):
    return os.path.basename(root_folder)

# Función para formatear las estadísticas
def format_statistics(stats, fmt=".10f"):
    return {key: [f"{val:{fmt}}" for val in values] for key, values in stats.items()}

# Función para asignar colores basados en el sufijo del archivo
def get_color_for_file(filename):
    if "negra" in filename:
        return "black"


# Función para generar los gráficos y los PDFs
def plot_comparison_grid(data_per_file, output_pdf_folder, output_img_folder, output_text_folder, lab, session, date, workbook):
    component_names = ["FRONT(A0)", "REAR(A1)", "LEFT(A2)", "RIGHT(A3)", "TOP(A4)", "BOTTOM(A5)"]
    session_name = "Vespertino" if session == "V" else "Matutino"
    sheet = workbook.active

    for filename, (timestamps, data, file_path) in data_per_file.items():
        color = get_color_for_file(filename)  # Color basado en el sufijo del archivo

        # Determinar las subcarpetas Baseline o Experimental Color
        if "mednegra.txt" in filename:
            subfolder = "Baseline"
        else:
            subfolder = "Experimental Color"

        pdf_file = os.path.join(output_pdf_folder, subfolder, f"{filename}.pdf")
        img_file = os.path.join(output_img_folder, subfolder, f"{filename}.png")

        with PdfPages(pdf_file) as pdf:
            if lab in n_change:
                n_lab = n_change[lab]
            fig, axes = plt.subplots(6, 3, figsize=(36, 45))  # Tamaño más grande de la figura
            fig.subplots_adjust(top=0.85, wspace=0.6)  # Más espacio entre columnas, se puede modificar wspace para ajustar el espacio
            fig.suptitle(f"Análisis del archivo: {filename}\n"
                         f"Laboratorio: {n_lab}, Turno: {session_name}, Fecha: {date}", fontsize=40, y=0.98)  # Título más grande
            axes = axes.flatten()
            stats = generate_statistics(data)  # Generar estadísticas una vez por archivo
            stats_formatted = format_statistics(stats, fmt=".10f")

            for i in range(6):
                # Gráfico de Datos Crudos
                ax = axes[3 * i]
                ax.plot(data[i], label='Datos', color=color)
                ax.set_title(f"Componente {component_names[i]} - Datos Crudos", fontsize=27)  # Títulos más grandes
                ax.set_xlabel("Registros", fontsize=25)
                ax.set_ylabel("Volts", fontsize=25)
                ax.legend()
                ax.tick_params(axis='x', rotation=45, labelsize=25)  # Tamaño de las etiquetas de los ejes x y y
                ax.tick_params(axis='y', labelsize=25)  # Tamaño de las etiquetas de los ejes y

                # Histograma de Derivadas
                derivative = np.gradient(data[i])
                ax_hist = axes[3 * i + 1]
                ax_hist.hist(derivative, bins=25, color=color, edgecolor='black')
                ax_hist.set_title(f"Componente {component_names[i]} - Histograma de Derivadas", fontsize=27)
                ax_hist.set_xlabel("Valor de la Derivada", fontsize=25)
                ax_hist.set_ylabel("Frecuencia", fontsize=25)
                ax_hist.tick_params(axis='x', labelsize=25)  # Tamaño de las etiquetas de los ejes x
                ax_hist.tick_params(axis='y', labelsize=25)  # Tamaño de las etiquetas de los ejes y

                # Texto de Estadísticas de Datos Crudos
                stats_text = (f"Media: {stats_formatted['mean'][i]}\n"
                              f"Mediana: {stats_formatted['median'][i]}\n"
                              f"Varianza: {stats_formatted['variance'][i]}\n"
                              f"Desviación Estándar: {stats_formatted['stdDev'][i]}\n"
                              f"Mínimo: {stats_formatted['min'][i]}\n"
                              f"Máximo: {stats_formatted['max'][i]}\n"
                              f"Primer Cuartil: {stats_formatted['firstQuartile'][i]}\n"
                              f"Tercer Cuartil: {stats_formatted['thirdQuartile'][i]}\n"
                              f"IQR: {stats_formatted['iqr'][i]}\n"
                              f"Varianza Recortada: {stats_formatted['trimmedVariance'][i]}\n"
                              f"Kurtosis: {stats_formatted['kurtosis'][i]}\n"
                              f"Asimetría: {stats_formatted['skewness'][i]}")
                ax_stats = axes[3 * i + 2]
                ax_stats.text(0.1, 0.5, stats_text, fontsize=25, verticalalignment='center')
                ax_stats.axis('off')
                ax_stats.set_title(f"Componente {component_names[i]} - Estadísticas de Datos Crudos", fontsize=24)

            plt.tight_layout(rect=[0, 0, 1, 0.95])  # Ajusta el espacio de toda la figura
            pdf.savefig(fig)  # Guarda el gráfico en el PDF
            plt.savefig(img_file, dpi=400)  # Guarda el gráfico como imagen de alta calidad
            plt.close()

            # Añadir datos al Excel
            add_data_to_excel(sheet, stats_formatted, date, session_name, filename, lab)

        # Generar archivo de texto con las estadísticas
        text_folder = os.path.join(output_text_folder, subfolder)
        stats_file = os.path.join(text_folder, f"{filename}.txt")
        os.makedirs(os.path.dirname(stats_file), exist_ok=True)
        with open(stats_file, 'w', encoding='utf-8') as f:
            f.write(f"{date},{session_name},{filename}\n\n")
            for i, component in enumerate(component_names):
                f.write(f"{component}\n")
                f.write(f"{stats_formatted['median'][i]},{stats_formatted['mean'][i]},{stats_formatted['variance'][i]},{stats_formatted['stdDev'][i]},{stats_formatted['min'][i]},{stats_formatted['max'][i]},{stats_formatted['firstQuartile'][i]},{stats_formatted['thirdQuartile'][i]},{stats_formatted['iqr'][i]},{stats_formatted['trimmedVariance'][i]},{stats_formatted['kurtosis'][i]},{stats_formatted['skewness'][i]}\n\n")

# Función para añadir datos al Excel
def add_data_to_excel(sheet, stats_formatted, date, session_name, filename, lab):
    component_names = ["FRONT(A0)", "REAR(A1)", "LEFT(A2)", "RIGHT(A3)", "TOP(A4)", "BOTTOM(A5)"]
    row_data = [date, session_name, lab, filename]  # Incluyendo el nombre del laboratorio

    for i in range(6):
        row_data.extend([
            stats_formatted['median'][i], stats_formatted['mean'][i], stats_formatted['variance'][i],
            stats_formatted['stdDev'][i], stats_formatted['min'][i], stats_formatted['max'][i],
            stats_formatted['firstQuartile'][i], stats_formatted['thirdQuartile'][i], stats_formatted['iqr'][i],
            stats_formatted['trimmedVariance'][i], stats_formatted['kurtosis'][i], stats_formatted['skewness'][i]
        ])
        row_data.append("")  # Espacio entre componentes

    sheet.append(row_data)

# Función para procesar todos los experimentos
def process_all_experiments(root_folder):
    date = extract_date_from_root_folder(root_folder)
    date = date.replace("- copia", "").strip()
    labs = ["Lab_Betta", "Lab_Gamma"]
    sessions = ["V", "M"]

    # Creando un nuevo libro de trabajo y hoja
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Datos"

    # Añadiendo la fila de títulos
    titles = [
        "DATE", "SHIFT", "LAB", "FILENAME",
        "General Statistics FRONT", "", "", "", "", "", "", "", "", "", "", "",
        "General Statistics REAR", "", "", "", "", "", "", "", "", "", "", "", "",
        "General Statistics LEFT", "", "", "", "", "", "", "", "", "", "", "", "",
        "General Statistics RIGHT", "", "", "", "", "", "", "", "", "", "", "", "",
        "General Statistics TOP", "", "", "", "", "", "", "", "", "", "", "", "",
        "General Statistics BOTTOM", "", "", "", "", "", "", "", "", "", "", "", ""
    ]
    sheet.append(titles)

    # Añadiendo subtítulos
    subtitles = [
        "", "", "", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", ""
    ]
    sheet.append(subtitles)

    for lab in labs:
        for session in sessions:
            experiment_folder = os.path.join(root_folder, f"{date} - {lab}", f"{date}.{session} - {lab}")
            print(f"Checking directory: {experiment_folder}")
            if os.path.exists(experiment_folder):
                output_pdf_folder = os.path.join(experiment_folder, "Data Analysis", "Graphics", "General Statistics", "PDFs")
                output_img_folder = os.path.join(experiment_folder, "Data Analysis", "Graphics", "General Statistics", "Images")
                output_text_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "General Statistics Text")
                os.makedirs(output_pdf_folder, exist_ok=True)
                os.makedirs(output_img_folder, exist_ok=True)
                os.makedirs(output_text_folder, exist_ok=True)

                # Crear subcarpetas Baseline y Experimental Color
                os.makedirs(os.path.join(output_pdf_folder, "Baseline"), exist_ok=True)
                os.makedirs(os.path.join(output_pdf_folder, "Experimental Color"), exist_ok=True)
                os.makedirs(os.path.join(output_img_folder, "Baseline"), exist_ok=True)
                os.makedirs(os.path.join(output_img_folder, "Experimental Color"), exist_ok=True)
                os.makedirs(os.path.join(output_text_folder, "Baseline"), exist_ok=True)
                os.makedirs(os.path.join(output_text_folder, "Experimental Color"), exist_ok=True)

                data_per_file = analyze_folder(experiment_folder)
                plot_comparison_grid(data_per_file, output_pdf_folder, output_img_folder, output_text_folder, lab, session, date, workbook)
            else:
                print(f"Directory does not exist: {experiment_folder}")

    # Ruta de salida del archivo Excel
    excel_output_path = os.path.join(root_folder, f"General_Statistics_{date}.xlsx")

    # Asegurarse de que el directorio existe antes de guardar
    os.makedirs(root_folder, exist_ok=True)

    # Guardando el archivo Excel al final de todos los experimentos
    workbook.save(excel_output_path)
    print("Archivo Excel generado exitosamente en:", excel_output_path)

# Cambia la ruta de root_folder a la que corresponde a tu directorio raíz

process_all_experiments(root_folder)

Checking directory: F:\Datos\20Sep24\20Sep24 - Lab_Betta\20Sep24.V - Lab_Betta
Directory does not exist: F:\Datos\20Sep24\20Sep24 - Lab_Betta\20Sep24.V - Lab_Betta
Checking directory: F:\Datos\20Sep24\20Sep24 - Lab_Betta\20Sep24.M - Lab_Betta
Directory does not exist: F:\Datos\20Sep24\20Sep24 - Lab_Betta\20Sep24.M - Lab_Betta
Checking directory: F:\Datos\20Sep24\20Sep24 - Lab_Gamma\20Sep24.V - Lab_Gamma
Directory does not exist: F:\Datos\20Sep24\20Sep24 - Lab_Gamma\20Sep24.V - Lab_Gamma
Checking directory: F:\Datos\20Sep24\20Sep24 - Lab_Gamma\20Sep24.M - Lab_Gamma
Archivo Excel generado exitosamente en: F:\Datos\20Sep24\General_Statistics_20Sep24.xlsx


## 2

$$
\Huge \text{Boxplots}
$$

In [41]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import iqr
import re

def read_sensor_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(', ')
                    data.append([float(num) for num in numbers])
        return timestamps, np.array(data).T
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return None, None

def calibrate_sensor(data, filename):
    try:
        results = {
            'mean': np.mean(data, axis=1),
            'median': np.median(data, axis=1),
            'variance': np.var(data, axis=1),
            'stdDev': np.std(data, axis=1),
            'min': np.min(data, axis=1),
            'max': np.max(data, axis=1),
            'firstQuartile': np.percentile(data, 25, axis=1),
            'thirdQuartile': np.percentile(data, 75, axis=1),
            'iqr': iqr(data, axis=1),
            'trimmedVariance': [np.var(np.trim_zeros(np.sort(sensor_data))) for sensor_data in data]
        }
        return results
    except Exception as e:
        print(f"Error al calcular estadísticas los datos, REVISAR ARCHIVO: {filename}: {e}")
        return None


def find_experiment_folder(root_folder, lab, session):
    # Expresión regular para buscar carpetas con formato de fecha
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    msg_error = 'no se halló la carpeta'
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and session in subfolder:
                    return os.path.join(root_folder, folder, subfolder)
    return msg_error


# Función para generar boxplots
def generate_boxplot(data, filename, output_folder, lab, session):
    labels = ["Frontal", "Rear", "Left", "Right", "Top", "Bottom"]
    color_dict = {
        'negra': 'black',
        'roja': 'red',
        'morada': 'purple',
        'azul': 'blue',
        'verde': 'green',
        'amarilla': 'yellow'
    }
    file_base_name = filename.split('.')[0]  # Remove extension
    color_suffix = file_base_name.split('med')[-1]
    color = color_dict.get(color_suffix, 'gray')  # Default color if not matched
    plt.figure(figsize=(10, 6))
    plt.boxplot(data.T, notch=True, patch_artist=True, boxprops=dict(facecolor=color))
    plt.title(f'Box Plot for {filename} - {lab} - {session}')
    plt.xlabel('Side of Cube')
    plt.ylabel('Volts')
    plt.xticks(range(1, len(labels) + 1), labels, rotation=45)
    plt.savefig(os.path.join(output_folder, f"{file_base_name}_{lab}_{session}_boxplot.png"))
    plt.close()


# Función para analizar las carpetas
def analyze_folder(folder_path, output_folder, lab, session):
    file_stats = {}
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if any(filename.startswith("0") and filename.endswith(ext) for ext in ["mednegra.txt", "medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt"]):
                file_path = os.path.join(root, filename)
                timestamps, data = read_sensor_data(file_path)
                if data is not None:
                    stats = calibrate_sensor(data, filename)
                    if stats is not None:
                        file_stats[filename] = {'stats': stats, 'data': data}
                        generate_boxplot(data, filename, output_folder, lab, session)
    return file_stats


# Función para procesar experimentos de un solo laboratorio
def process_lab_experiments(root_folder, lab):
    sessions = ["V", "M"]
    all_data = {session: {} for session in sessions}
    global_output_folder = os.path.join(root_folder, "Data Analysis", "Global Comparisons", lab)
    os.makedirs(global_output_folder, exist_ok=True)

    for session in sessions:
        experiment_folder = find_experiment_folder(root_folder, lab, session)
        if os.path.exists(experiment_folder):
            output_folder_base = os.path.join(experiment_folder, "Data Analysis", "Graphics")
            function_title = "Boxplots"
            output_folder = os.path.join(output_folder_base, function_title)
            os.makedirs(output_folder, exist_ok=True)
            file_stats = analyze_folder(experiment_folder, output_folder, lab, session)
            all_data[session] = file_stats

    return all_data


# Función para procesar ambos laboratorios
def process_all_experiments(root_folder):
    labs = ["Lab_Betta", "Lab_Gamma"]
    all_data = {}
    
    for lab in labs:
        all_data[lab] = process_lab_experiments(root_folder, lab)

    # Generar comparaciones entre los laboratorios
    for session in ["V", "M"]:
        for filename in all_data["Lab_Betta"][session]:
            if filename in all_data["Lab_Gamma"][session]:
                data_a = all_data["Lab_Betta"][session][filename]['data']
                data_b = all_data["Lab_Gamma"][session][filename]['data']
                color_suffix_a = filename.split('med')[-1].split('.')[0]
                color_suffix_b = filename.split('med')[-1].split('.')[0]
                color_a = color_dict.get(color_suffix_a, 'blue')
                color_b = color_dict.get(color_suffix_b, 'green')
                generate_comparison_boxplot(data_a, data_b, filename, global_output_folder, session, color_a, color_b)


# Procesar todos los experimentos
process_all_experiments(root_folder)

## 3
$$
\Huge \text{First Integer Helicity Betta vs Gamma}
$$

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import os
import glob
import re

# Triadas definidas
triadas = ["FRT", "PLB", "FLT", "PRB", "FRB", "PLT", "FLB", "PRT", "RTB", "FLP", "LTB", "FRP"]

def read_data(filepath):
    timestamps = []
    values = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                    timestamps.append(timestamp)
                    values.append(float(parts[1]))
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return timestamps, np.array(values)

def get_color(filename):
    color_map = {
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

def plot_pair(baseline_file, experimental_file, output_dir, lab, shift, date_str):
    timestamps_baseline, baseline_values = read_data(baseline_file)
    timestamps_experimental, experimental_values = read_data(experimental_file)

    if len(timestamps_baseline) == 0 or len(timestamps_experimental) == 0:
        print(f"Data missing for {baseline_file} or {experimental_file}. Skipping.")
        return

    # Reiniciar todas las estampas de tiempo
    start_time_baseline = timestamps_baseline[0]
    start_time_experimental = timestamps_experimental[0]
    timestamps_baseline = [(ts - start_time_baseline + datetime.datetime(1900, 1, 1)) for ts in timestamps_baseline]
    timestamps_experimental = [(ts - start_time_experimental + datetime.datetime(1900, 1, 1)) for ts in timestamps_experimental]

    fig, ax = plt.subplots(figsize=(10, 8))
    baseline_times = mdates.date2num(timestamps_baseline)
    experimental_times = mdates.date2num(timestamps_experimental)
    
    ax.plot(baseline_times, baseline_values, c='black', label=os.path.basename(baseline_file))
    ax.plot(experimental_times, experimental_values, c=get_color(experimental_file), label=os.path.basename(experimental_file))
    ax.set_title(f'Helicity Comparison - Lab: {lab} - Shift: {"Vespertino" if shift == "V" else "Matutino"} - Date: {date_str}', fontsize=14)
    ax.set_xlabel('Time')
    ax.set_ylabel('Helicity')
    ax.legend()
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.gcf().autofmt_xdate()
    plt.tight_layout()

    # Generar un nombre único para cada gráfico
    baseline_name = os.path.splitext(os.path.basename(baseline_file))[0]
    experimental_name = os.path.splitext(os.path.basename(experimental_file))[0]
    output_filename = f"{baseline_name}_VS_{experimental_name}.png"

    # Determinar la carpeta de salida en función del número de triada
    triad_number = re.search(r'helicity_([A-Z]{3})', experimental_file)  # Extraer letras mayúsculas
    if triad_number:
        triad_number = triad_number.group(1)  # Obtener las letras mayúsculas
    else:
        print(f"No se pudo extraer el número de triada de {experimental_file}. Skipping.")
        return  # Si no se encuentra el número, salir de la función

    # Crear la subcarpeta para la triada
    triad_folder = os.path.join(output_dir, f"{triad_number}")  # Usar las letras mayúsculas
    if not os.path.exists(triad_folder):
        os.makedirs(triad_folder)

    output_filepath = os.path.join(triad_folder, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files):
    combinations = []
    # Diccionarios para almacenar los archivos por el número extraído
    baseline_dict = {}
    experimental_dict = {}

    # Extraer números de los archivos de línea base
    for f in baseline_files:
        match = re.search(r'velocity_(\d+)', f)
        if match:
            number = match.group(1)
            baseline_dict[number] = f  # Almacenar la ruta del archivo por número

    # Extraer números de los archivos experimentales
    for f in experimental_files:
        match = re.search(r'velocity_(\d+)', f)
        if match:
            number = match.group(1)
            experimental_dict[number] = f  # Almacenar la ruta del archivo por número

    # Generar combinaciones solo para números comunes
    common_numbers = baseline_dict.keys() & experimental_dict.keys()  # Intersección de números

    for number in common_numbers:
        combinations.append((baseline_dict[number], experimental_dict[number]))

    return combinations

def main():
    global root_folder
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]
    
    for lab in labs:
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if not lab_dirs:
                print(f"No se encontró ningún directorio para {lab}.")
                continue
            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Helicity_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Helicity_Experimental_Color")
            output_directory = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Graphics", "Sum_Helicity_Graph_Color_vs_Baseline")

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            # Buscar archivos en las subcarpetas para los archivos de línea base
            baseline_files = []
            for root, dirs, files in os.walk(baseline_dir):
                for f in files:
                    if f.endswith('mednegra.txt') and "00" not in f:
                        baseline_files.append(os.path.join(root, f))

            # Buscar archivos en las subcarpetas para los archivos experimentales
            experimental_files = []
            for root, dirs, files in os.walk(exp_dir):
                for f in files:
                    if any(f.endswith(suffix) for suffix in ['medroja.txt', 'medmorada.txt', 'medazul.txt', 'medverde.txt', 'medamarilla.txt']) and "00" not in f:
                        experimental_files.append(os.path.join(root, f))

            for triad in triadas:
                # Filtrar los archivos por triada
                baseline_files_filtered = [f for f in baseline_files if triad in f]
                experimental_files_filtered = [f for f in experimental_files if triad in f]

                combinations = generate_combinations(baseline_files_filtered, experimental_files_filtered)

                for base_file, exp_file in combinations:
                    if os.path.exists(base_file) and os.path.exists(exp_file):
                        plot_pair(base_file, exp_file, output_directory, lab, shift, date_str)

if __name__ == "__main__":
    main()

No se encontró ningún directorio para Lab_Betta.
No se encontró ningún directorio para Lab_Betta.
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Helicity_Graph_Color_vs_Baseline\FRT\sum_helicity_FRT_velocity_01mednegra_VS_sum_helicity_FRT_velocity_01medamarilla.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Helicity_Graph_Color_vs_Baseline\FRT\sum_helicity_FRT_velocity_04mednegra_VS_sum_helicity_FRT_velocity_04medroja.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Helicity_Graph_Color_vs_Baseline\FRT\sum_helicity_FRT_velocity_03mednegra_VS_sum_helicity_FRT_velocity_03medroja.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Helicity_Graph_Color_vs_Baseline\FRT\sum_helicity_FRT_velocity_02mednegra_VS_sum




## 4 
$$
\Huge \text{Sum Absolute Helicity Betta vs Gamma}
$$


In [43]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import os
import glob
import re

# Triadas definidas
triadas = ["FRT", "PLB", "FLT", "PRB", "FRB", "PLT", "FLB", "PRT", "RTB", "FLP", "LTB", "FRP"]

def read_data(filepath):
    timestamps = []
    values = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                    timestamps.append(timestamp)
                    values.append(float(parts[1]))
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return timestamps, np.array(values)

def get_color(filename):
    color_map = {
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

def plot_pair(baseline_file, experimental_file, output_dir, lab, shift, date_str):
    timestamps_baseline, baseline_values = read_data(baseline_file)
    timestamps_experimental, experimental_values = read_data(experimental_file)

    if len(timestamps_baseline) == 0 or len(timestamps_experimental) == 0:
        print(f"Data missing for {baseline_file} or {experimental_file}. Skipping.")
        return

    # Reiniciar todas las estampas de tiempo
    start_time_baseline = timestamps_baseline[0]
    start_time_experimental = timestamps_experimental[0]
    timestamps_baseline = [(ts - start_time_baseline + datetime.datetime(1900, 1, 1)) for ts in timestamps_baseline]
    timestamps_experimental = [(ts - start_time_experimental + datetime.datetime(1900, 1, 1)) for ts in timestamps_experimental]

    fig, ax = plt.subplots(figsize=(10, 8))
    baseline_times = mdates.date2num(timestamps_baseline)
    experimental_times = mdates.date2num(timestamps_experimental)
    
    ax.plot(baseline_times, baseline_values, c='black', label=os.path.basename(baseline_file))
    ax.plot(experimental_times, experimental_values, c=get_color(experimental_file), label=os.path.basename(experimental_file))
    ax.set_title(f'Helicity Comparison - Lab: {lab} - Shift: {"Vespertino" if shift == "V" else "Matutino"} - Date: {date_str}', fontsize=14)
    ax.set_xlabel('Time')
    ax.set_ylabel('Helicity')
    ax.legend()
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.gcf().autofmt_xdate()
    plt.tight_layout()

    # Generar un nombre único para cada gráfico
    baseline_name = os.path.splitext(os.path.basename(baseline_file))[0]
    experimental_name = os.path.splitext(os.path.basename(experimental_file))[0]
    output_filename = f"{baseline_name}_VS_{experimental_name}.png"

    # Determinar la carpeta de salida en función del número de triada
    triad_number = re.search(r'helicity_([A-Z]{3})', experimental_file)  # Extraer letras mayúsculas
    if triad_number:
        triad_number = triad_number.group(1)  # Obtener las letras mayúsculas
    else:
        print(f"No se pudo extraer el número de triada de {experimental_file}. Skipping.")
        return  # Si no se encuentra el número, salir de la función

    # Crear la subcarpeta para la triada
    triad_folder = os.path.join(output_dir, f"{triad_number}")  # Usar las letras mayúsculas
    if not os.path.exists(triad_folder):
        os.makedirs(triad_folder)

    output_filepath = os.path.join(triad_folder, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files):
    combinations = []
    # Diccionarios para almacenar los archivos por el número extraído
    baseline_dict = {}
    experimental_dict = {}

    # Extraer números de los archivos de línea base
    for f in baseline_files:
        match = re.search(r'velocity_(\d+)', f)
        if match:
            number = match.group(1)
            baseline_dict[number] = f  # Almacenar la ruta del archivo por número

    # Extraer números de los archivos experimentales
    for f in experimental_files:
        match = re.search(r'velocity_(\d+)', f)
        if match:
            number = match.group(1)
            experimental_dict[number] = f  # Almacenar la ruta del archivo por número

    # Generar combinaciones solo para números comunes
    common_numbers = baseline_dict.keys() & experimental_dict.keys()  # Intersección de números

    for number in common_numbers:
        combinations.append((baseline_dict[number], experimental_dict[number]))

    return combinations

def main():
    global root_folder
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]
    
    for lab in labs:
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if not lab_dirs:
                print(f"No se encontró ningún directorio para {lab}.")
                continue
            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Experimental_Color")
            output_directory = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Graphics", "Sum_Absolute_Helicity_Graph_Color_vs_Baseline")

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            # Buscar archivos en las subcarpetas para los archivos de línea base
            baseline_files = []
            for root, dirs, files in os.walk(baseline_dir):
                for f in files:
                    if f.endswith('mednegra.txt') and "00" not in f:
                        baseline_files.append(os.path.join(root, f))

            # Buscar archivos en las subcarpetas para los archivos experimentales
            experimental_files = []
            for root, dirs, files in os.walk(exp_dir):
                for f in files:
                    if any(f.endswith(suffix) for suffix in ['medroja.txt', 'medmorada.txt', 'medazul.txt', 'medverde.txt', 'medamarilla.txt']) and "00" not in f:
                        experimental_files.append(os.path.join(root, f))

            for triad in triadas:
                # Filtrar los archivos por triada
                baseline_files_filtered = [f for f in baseline_files if triad in f]
                experimental_files_filtered = [f for f in experimental_files if triad in f]

                combinations = generate_combinations(baseline_files_filtered, experimental_files_filtered)

                for base_file, exp_file in combinations:
                    if os.path.exists(base_file) and os.path.exists(exp_file):
                        plot_pair(base_file, exp_file, output_directory, lab, shift, date_str)

if __name__ == "__main__":
    main()

No se encontró ningún directorio para Lab_Betta.
No se encontró ningún directorio para Lab_Betta.
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Absolute_Helicity_Graph_Color_vs_Baseline\FRT\sum_absolute_helicity_FRT_velocity_01mednegra_VS_sum_absolute_helicity_FRT_velocity_01medamarilla.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Absolute_Helicity_Graph_Color_vs_Baseline\FRT\sum_absolute_helicity_FRT_velocity_04mednegra_VS_sum_absolute_helicity_FRT_velocity_04medroja.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Absolute_Helicity_Graph_Color_vs_Baseline\FRT\sum_absolute_helicity_FRT_velocity_03mednegra_VS_sum_absolute_helicity_FRT_velocity_03medroja.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum


## 5 
$$
\Huge \text{First Integer Enstrophy Betta vs Gamma}
$$

In [44]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import os
import glob
import re

# Triadas definidas
triadas = ["FRT", "PLB", "FLT", "PRB", "FRB", "PLT", "FLB", "PRT", "RTB", "FLP", "LTB", "FRP"]

def read_data(filepath):
    timestamps = []
    values = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                    timestamps.append(timestamp)
                    values.append(float(parts[1]))
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return timestamps, np.array(values)

def get_color(filename):
    color_map = {
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

def plot_pair(baseline_file, experimental_file, output_dir, lab, shift, date_str):
    timestamps_baseline, baseline_values = read_data(baseline_file)
    timestamps_experimental, experimental_values = read_data(experimental_file)

    if len(timestamps_baseline) == 0 or len(timestamps_experimental) == 0:
        print(f"Data missing for {baseline_file} or {experimental_file}. Skipping.")
        return

    # Reiniciar todas las estampas de tiempo
    start_time_baseline = timestamps_baseline[0]
    start_time_experimental = timestamps_experimental[0]
    timestamps_baseline = [(ts - start_time_baseline + datetime.datetime(1900, 1, 1)) for ts in timestamps_baseline]
    timestamps_experimental = [(ts - start_time_experimental + datetime.datetime(1900, 1, 1)) for ts in timestamps_experimental]

    fig, ax = plt.subplots(figsize=(10, 8))
    baseline_times = mdates.date2num(timestamps_baseline)
    experimental_times = mdates.date2num(timestamps_experimental)
    
    ax.plot(baseline_times, baseline_values, c='black', label=os.path.basename(baseline_file))
    ax.plot(experimental_times, experimental_values, c=get_color(experimental_file), label=os.path.basename(experimental_file))
    ax.set_title(f'Helicity Comparison - Lab: {lab} - Shift: {"Vespertino" if shift == "V" else "Matutino"} - Date: {date_str}', fontsize=14)
    ax.set_xlabel('Time')
    ax.set_ylabel('Helicity')
    ax.legend()
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.gcf().autofmt_xdate()
    plt.tight_layout()

    # Generar un nombre único para cada gráfico
    baseline_name = os.path.splitext(os.path.basename(baseline_file))[0]
    experimental_name = os.path.splitext(os.path.basename(experimental_file))[0]
    output_filename = f"{baseline_name}_VS_{experimental_name}.png"

    # Determinar la carpeta de salida en función del número de triada
    triad_number = re.search(r'vorticity_([A-Z]{3})', experimental_file)  # Extraer letras mayúsculas
    if triad_number:
        triad_number = triad_number.group(1)  # Obtener las letras mayúsculas
    else:
        print(f"No se pudo extraer el número de triada de {experimental_file}. Skipping.")
        return  # Si no se encuentra el número, salir de la función

    # Crear la subcarpeta para la triada
    triad_folder = os.path.join(output_dir, f"{triad_number}")  # Usar las letras mayúsculas
    if not os.path.exists(triad_folder):
        os.makedirs(triad_folder)

    output_filepath = os.path.join(triad_folder, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files):
    combinations = []
    # Diccionarios para almacenar los archivos por el número extraído
    baseline_dict = {}
    experimental_dict = {}

    # Extraer números de los archivos de línea base
    for f in baseline_files:
        match = re.search(r'velocity_(\d+)', f)
        if match:
            number = match.group(1)
            baseline_dict[number] = f  # Almacenar la ruta del archivo por número

    # Extraer números de los archivos experimentales
    for f in experimental_files:
        match = re.search(r'velocity_(\d+)', f)
        if match:
            number = match.group(1)
            experimental_dict[number] = f  # Almacenar la ruta del archivo por número

    # Generar combinaciones solo para números comunes
    common_numbers = baseline_dict.keys() & experimental_dict.keys()  # Intersección de números

    for number in common_numbers:
        combinations.append((baseline_dict[number], experimental_dict[number]))

    return combinations

def main():
    global root_folder
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]
    
    for lab in labs:
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if not lab_dirs:
                print(f"No se encontró ningún directorio para {lab}.")
                continue
            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Experimental_Color")
            output_directory = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Graphics", "Sum_Enstrophy_Graph_Color_vs_Baseline")

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            # Buscar archivos en las subcarpetas para los archivos de línea base
            baseline_files = []
            for root, dirs, files in os.walk(baseline_dir):
                for f in files:
                    if f.endswith('mednegra.txt') and "00" not in f:
                        baseline_files.append(os.path.join(root, f))

            # Buscar archivos en las subcarpetas para los archivos experimentales
            experimental_files = []
            for root, dirs, files in os.walk(exp_dir):
                for f in files:
                    if any(f.endswith(suffix) for suffix in ['medroja.txt', 'medmorada.txt', 'medazul.txt', 'medverde.txt', 'medamarilla.txt']) and "00" not in f:
                        experimental_files.append(os.path.join(root, f))

            for triad in triadas:
                # Filtrar los archivos por triada
                baseline_files_filtered = [f for f in baseline_files if triad in f]
                experimental_files_filtered = [f for f in experimental_files if triad in f]

                combinations = generate_combinations(baseline_files_filtered, experimental_files_filtered)

                for base_file, exp_file in combinations:
                    if os.path.exists(base_file) and os.path.exists(exp_file):
                        plot_pair(base_file, exp_file, output_directory, lab, shift, date_str)

if __name__ == "__main__":
    main()

No se encontró ningún directorio para Lab_Betta.
No se encontró ningún directorio para Lab_Betta.
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Enstrophy_Graph_Color_vs_Baseline\FRT\sum_enstrophy_vorticity_FRT_velocity_01mednegra_VS_sum_enstrophy_vorticity_FRT_velocity_01medamarilla.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Enstrophy_Graph_Color_vs_Baseline\FRT\sum_enstrophy_vorticity_FRT_velocity_04mednegra_VS_sum_enstrophy_vorticity_FRT_velocity_04medroja.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Enstrophy_Graph_Color_vs_Baseline\FRT\sum_enstrophy_vorticity_FRT_velocity_03mednegra_VS_sum_enstrophy_vorticity_FRT_velocity_03medroja.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Sum_Enstrophy_G

## 6


$$
    \Huge \text{3D Curl Comparisons }
$$

In [45]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import datetime
import os
import glob
from matplotlib.animation import FuncAnimation

# Función para leer datos del sensor
def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                try:
                    timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                    timestamps.append(timestamp)
                    values.append([float(num) for num in parts[1].strip('()').split(',')])
                except ValueError as e:
                    print(f"Error processing line in file {filepath}: {line}\n{e}")
    return timestamps, np.array(values)

# Función para comparar y graficar
def plot_comparison(baseline_file, exp_file, color, output_directory, lab, shift, date_str):
    try:
        _, baseline_values = read_data(baseline_file)
        _, exp_values = read_data(exp_file)
    except Exception as e:
        print(f"Error reading data files:\n{e}")
        return

    shift_label = "Vespertino" if shift == "V" else "Matutino"

    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(baseline_values[:, 0], baseline_values[:, 1], baseline_values[:, 2], c='black', label=os.path.basename(baseline_file), s=50, alpha=0.6)
    ax.scatter(exp_values[:, 0], exp_values[:, 1], exp_values[:, 2], c=color, label=os.path.basename(exp_file), s=50, alpha=0.6)
    if lab in n_change:
        n_lab = n_change[lab]
    ax.set_title(f'3D Comparison Curl - {n_lab} - {shift_label} Shift - Date: {date_str}', fontsize=20)
    ax.set_xlabel('X', fontsize=15)
    ax.set_ylabel('Y', fontsize=15)
    ax.set_zlabel('Z', fontsize=15)

    legend = ax.legend(fontsize=24, loc='upper left', frameon=True, framealpha=0.9)
    for handle in legend.legend_handles:
        handle.set_sizes([300])

    def update(frame):
        ax.view_init(elev=10, azim=frame)
        return fig,

    output_filename = f"{os.path.basename(baseline_file).replace('mednegra', 'comparison')}_{os.path.basename(exp_file)}"
    output_filepath = os.path.join(output_directory, output_filename)

    ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), interval=100, blit=False)
    ani.save(f'{output_filepath}.gif', writer='pillow', fps=10)
    plt.close(fig)

# Función para extraer los números de archivo
def extract_file_numbers(filenames):
    file_numbers = sorted([int(f.split('med')[0][-2:]) for f in filenames if f[-6:] != '00.txt'])
    return file_numbers

# Función para generar combinaciones de archivos
def generate_combinations(baseline_files, experimental_files):
    combinations = []
    for base_file in baseline_files:
        base_num = int(base_file.split('med')[0][-2:])
        for exp_file in experimental_files:
            exp_num = int(exp_file.split('med')[0][-2:])
            if exp_num == base_num or exp_num == base_num - 1:
                combinations.append((base_file, exp_file))
    return combinations

# Función principal para procesar los archivos
def main(root_folder):
    date_str = os.path.basename(root_folder)
    date_str= date_str.replace("- copia", "").strip()
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    color_map = {
        "medroja.txt": "red",
        "medmorada.txt": "purple",
        "medazul.txt": "blue",
        "medverde.txt": "green",
        "medamarilla.txt": "yellow"
    }

    for lab in labs:
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if not lab_dirs:
                print(f"No se encontró ningún directorio para {lab}.")
                continue
            lab_dir = lab_dirs[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Curl_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Curl_Experimental_Color")
            output_directory = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Graphics", "Curl_Graph_Color_vs_Baseline")

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(baseline_dir)
                print(exp_dir)
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            baseline_files = [f for f in os.listdir(baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]
            experimental_files = [f for f in os.listdir(exp_dir) if any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]

            combinations = generate_combinations(baseline_files, experimental_files)

            for base_file, exp_file in combinations:
                baseline_file = os.path.join(baseline_dir, base_file)
                experimental_file = os.path.join(exp_dir, exp_file)
                if os.path.exists(baseline_file) and os.path.exists(experimental_file):
                    exp_suffix = ''.join([i for i in os.path.basename(experimental_file).split('curl_')[-1] if not i.isdigit()])
                    color = color_map.get(exp_suffix, 'black')
                    plot_comparison(baseline_file, experimental_file, color, output_directory, lab, shift, date_str)

if __name__ == "__main__":

    main(root_folder)


No se encontró ningún directorio para Lab_Betta.
No se encontró ningún directorio para Lab_Betta.
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20Sep24.V - Lab_Gamma\Data Analysis\Processing Data\Curl_Baseline
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20Sep24.V - Lab_Gamma\Data Analysis\Processing Data\Curl_Experimental_Color
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.


## 7

$$
 \Huge \text{Individual 3D Curl Baseline vs Experimental Color}
$$

In [57]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import datetime
import os
import glob
from matplotlib.animation import FuncAnimation

# Diccionario con los nombres de las triadas
triad_names = {
    "FRT": "Frontal-Derecho-Superior",
    "PLB": "Trasero-Izquierdo-Inferior",
    "FLT": "Frontal-Izquierdo-Superior",
    "PRB": "Trasero-Derecho-Inferior",
    "FRB": "Frontal-Derecho-Inferior",
    "PLT": "Trasero-Izquierdo-Superior",
    "FLB": "Frontal-Izquierdo-Inferior",
    "PRT": "Trasero-Derecho-Superior",
    "RTB": "Derecho-Superior-Inferior",
    "FLP": "Frontal-Izquierdo-Trasero",
    "LTB": "Izquierdo-Superior-Inferior",
    "FRP": "Frontal-Derecho-Trasero"
}

# Función para leer datos del sensor
def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                try:
                    timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                    timestamps.append(timestamp)
                    values.append([float(num) for num in parts[1].strip('()').split(',')])
                except ValueError as e:
                    print(f"Error processing line in file {filepath}: {line}\n{e}")
    return timestamps, np.array(values)

# Función para extraer el código de triada desde el nombre del archivo
def extract_triad_code(filename):
    for code in triad_names:
        if code in filename:
            return code
    return "Desconocido"

# Función para comparar y graficar los laboratorios
def plot_comparison_labs(baseline_file_betta, exp_file_betta, color_betta, baseline_file_gamma, exp_file_gamma, color_gamma, output_directory, shift, date_str):
    try:
        _, baseline_values_betta = read_data(baseline_file_betta)
        _, exp_values_betta = read_data(exp_file_betta)
        _, baseline_values_gamma = read_data(baseline_file_gamma)
        _, exp_values_gamma = read_data(exp_file_gamma)
    except Exception as e:
        print(f"Error reading data files:\n{e}")
        return

    shift_label = "Vespertino" if shift == "V" else "Matutino"
    triad_code_betta = extract_triad_code(baseline_file_betta)
    triad_code_gamma = extract_triad_code(baseline_file_gamma)

    triad_name_betta = triad_names.get(triad_code_betta, "Desconocido")
    triad_name_gamma = triad_names.get(triad_code_gamma, "Desconocido")

    fig = plt.figure(figsize=(24, 10))

    ax_betta = fig.add_subplot(121, projection='3d')
    ax_betta.scatter(baseline_values_betta[:, 0], baseline_values_betta[:, 1], baseline_values_betta[:, 2], c='black', label=os.path.basename(baseline_file_betta), s=50, alpha=0.6)
    ax_betta.scatter(exp_values_betta[:, 0], exp_values_betta[:, 1], exp_values_betta[:, 2], c=color_betta, label=os.path.basename(exp_file_betta), s=50, alpha=0.6)
    ax_betta.set_title(f'Lab Betta - {shift_label} Shift - {triad_name_betta} - Date: {date_str}', fontsize=20)
    ax_betta.set_xlabel('X', fontsize=15)
    ax_betta.set_ylabel('Y', fontsize=15)
    ax_betta.set_zlabel('Z', fontsize=15)

    ax_gamma = fig.add_subplot(122, projection='3d')
    ax_gamma.scatter(baseline_values_gamma[:, 0], baseline_values_gamma[:, 1], baseline_values_gamma[:, 2], c='black', label=os.path.basename(baseline_file_gamma), s=50, alpha=0.6)
    ax_gamma.scatter(exp_values_gamma[:, 0], exp_values_gamma[:, 1], exp_values_gamma[:, 2], c=color_gamma, label=os.path.basename(exp_file_gamma), s=50, alpha=0.6)
    ax_gamma.set_title(f'Lab Gamma - {shift_label} Shift - {triad_name_gamma} - Date: {date_str}', fontsize=20)
    ax_gamma.set_xlabel('X', fontsize=15)
    ax_gamma.set_ylabel('Y', fontsize=15)
    ax_gamma.set_zlabel('Z', fontsize=15)

    def update(frame):
        ax_betta.view_init(elev=10, azim=frame)
        ax_gamma.view_init(elev=10, azim=frame)
        return fig,

    output_filename = f"{os.path.basename(baseline_file_betta).replace('mednegra', 'comparison_betta_vs_gamma')}_{os.path.basename(exp_file_betta)}"
    output_filepath = os.path.join(output_directory, output_filename)

    ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), interval=100, blit=False)
    ani.save(f'{output_filepath}.gif', writer='pillow', fps=10)
    plt.close(fig)

# Función principal para procesar los archivos de ambos laboratorios
def main(root_folder):
    date_str = os.path.basename(root_folder)
    date_str = date_str.replace("-2", "").strip()
    shifts = ["V", "M"]

    color_map = {
        "medroja.txt": "red",
        "medmorada.txt": "purple",
        "medazul.txt": "blue",
        "medverde.txt": "green",
        "medamarilla.txt": "yellow"
    }

    for shift in shifts:
        lab_betta_dirs = glob.glob(os.path.join(root_folder, f"*Lab_Betta"))
        lab_gamma_dirs = glob.glob(os.path.join(root_folder, f"*Lab_Gamma"))

        if not lab_betta_dirs:
            print("No se encontró el directorio para Lab Betta.")
            continue
        if not lab_gamma_dirs:
            print("No se encontró el directorio para Lab Gamma.")
            continue

        lab_betta_dir = lab_betta_dirs[0]
        lab_gamma_dir = lab_gamma_dirs[0]

        betta_baseline_dir = os.path.join(lab_betta_dir, f"{date_str}.{shift} - Lab_Betta", "Data Analysis", "Processing Data", "Curl_Baseline")
        betta_exp_dir = os.path.join(lab_betta_dir, f"{date_str}.{shift} - Lab_Betta", "Data Analysis", "Processing Data", "Curl_Experimental_Color")

        gamma_baseline_dir = os.path.join(lab_gamma_dir, f"{date_str}.{shift} - Lab_Gamma", "Data Analysis", "Processing Data", "Curl_Baseline")
        gamma_exp_dir = os.path.join(lab_gamma_dir, f"{date_str}.{shift} - Lab_Gamma", "Data Analysis", "Processing Data", "Curl_Experimental_Color")

        output_directory = os.path.join(root_folder, "Data Analysis", "3D Curl Comparissons")
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        # Omitir si alguna de las carpetas requeridas no existe
        if not os.path.exists(betta_baseline_dir) or not os.path.exists(betta_exp_dir) or not os.path.exists(gamma_baseline_dir) or not os.path.exists(gamma_exp_dir):
            print(f"Omitiendo cálculo para el turno {shift} ya que uno o más directorios no existen.")
            continue

        betta_baseline_files = [f for f in os.listdir(betta_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]
        gamma_baseline_files = [f for f in os.listdir(gamma_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]

        betta_exp_files = [f for f in os.listdir(betta_exp_dir) if f.endswith('.txt') and not f.endswith('mednegra.txt') and "00" not in f]
        gamma_exp_files = [f for f in os.listdir(gamma_exp_dir) if f.endswith('.txt') and not f.endswith('mednegra.txt') and "00" not in f]

        for betta_baseline in betta_baseline_files:
            for gamma_baseline in gamma_baseline_files:
                for betta_exp in betta_exp_files:
                    for gamma_exp in gamma_exp_files:
                        print(f"Procesando: {betta_baseline}, {betta_exp} vs {gamma_baseline}, {gamma_exp} en el turno {shift}.")
                        plot_comparison_labs(
                            os.path.join(betta_baseline_dir, betta_baseline),
                            os.path.join(betta_exp_dir, betta_exp),
                            color_map.get(betta_exp, 'black'),
                            os.path.join(gamma_baseline_dir, gamma_baseline),
                            os.path.join(gamma_exp_dir, gamma_exp),
                            color_map.get(gamma_exp, 'black'),
                            output_directory,
                            shift,
                            date_str
                        )

if __name__ == "__main__":
    main(root_folder)


No se encontró el directorio para Lab Betta.
No se encontró el directorio para Lab Betta.


## 8

$$
  \Huge \text{Velocity Field  Baseline  Nivel}
$$

In [56]:
# Curvas de Nivel Experimentales
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import glob
import re
import datetime

# Diccionario de triadas y códigos
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

def extract_number(filename):
    match = re.search(r'velocity_(\d+)med', filename)
    if match:
        return match.group(1)
    return None

def read_data(filepath):
    """
    Extrae datos del archivo y los devuelve como arrays de x, y, z.
    """
    timestamps = []
    values_x = []
    values_y = []
    values_z = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                # Parsear el timestamp
                timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                timestamps.append(timestamp)

                # Remover paréntesis antes de convertir a float
                coords_str = parts[1].strip().replace('(', '').replace(')', '')
                coords = list(map(float, coords_str.split(',')))  # Convertir a float
                values_x.append(coords[0])
                values_y.append(coords[1])
                values_z.append(coords[2])
    
    return np.array(values_x), np.array(values_y), np.array(values_z)

def plot_contour(x, y, z, output_file):
    """
    Genera y guarda la gráfica de curvas de nivel.
    """
    # Crear una malla de puntos (grid) para la interpolación
    grid_x, grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]

    # Interpolación de los valores de z en la malla
    grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

    # Crear la gráfica de curvas de nivel
    plt.figure(figsize=(10, 8))
    contour = plt.contourf(grid_x, grid_y, grid_z, cmap='viridis')
    plt.colorbar(contour)
    plt.title('Curvas de Nivel')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Guardar la gráfica
    plt.savefig(output_file)
    plt.close()

def process_files(file_list, output_dir):
    """
    Procesa una lista de archivos, genera y guarda gráficos de curvas de nivel para cada uno.
    """
    for file_path in file_list:
        print(file_path)
        x, y, z = read_data(file_path)
        triada = next((code for code, value in triad_codes.items() if value in os.path.basename(file_path)), None)
        if not triada:
            continue

        triada_code = triad_codes.get(triada)
        output_dir_triade = os.path.join(output_dir, triada_code)

        if not os.path.exists(output_dir_triade):
            os.makedirs(output_dir_triade)

        file_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = os.path.join(output_dir_triade, f"{file_name}_contour.png")
        plot_contour(x, y, z, output_file)

def main():
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    for shift in shifts:
        lab_dirs = []
        for lab in labs:
            lab_dir = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if lab_dir:  # Verifica si se encontró al menos un directorio
                lab_dirs.append(lab_dir[0])

        print(lab_dirs)

        if not lab_dirs:  # Verifica si no se encontraron directorios
            print("No se encontraron directorios para uno o ambos laboratorios.")
            continue

        for lab_dir in lab_dirs:
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Velocity_Baseline")
            
            output_directory_baseline = os.path.join(root_folder, "Data Analysis", "Graphics", "Velocity_Baseline_Nivel", lab)

            if not os.path.exists(baseline_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory_baseline):
                os.makedirs(output_directory_baseline)

            # Buscar archivos en directorios de triadas
            baseline_files = []

            for triad_code in triad_codes.values():
                baseline_files.extend(glob.glob(os.path.join(baseline_dir, triad_code, 'velocity_*med*.txt')))

            # Procesar archivos de línea base
            process_files(baseline_files, output_directory_baseline)

if __name__ == "__main__":
    main()

['F:\\Datos\\20Sep24\\20sep24 - Lab_Gamma']
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.
['F:\\Datos\\20Sep24\\20sep24 - Lab_Gamma']
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Baseline\FRT\Velocity_FRT_velocity_01mednegra.txt
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Baseline\FRT\Velocity_FRT_velocity_02mednegra.txt
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Baseline\FRT\Velocity_FRT_velocity_03mednegra.txt
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Baseline\FRT\Velocity_FRT_velocity_04mednegra.txt
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Baseline\FRT\Velocity_FRT_velocity_05mednegra.txt
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Baselin

## 9

$$
  \Huge \text{Velocity Field  MEI  Nivel}
$$

In [63]:
# Curvas de Nivel Experimentales
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import glob
import re
import datetime

# Diccionario de triadas y códigos
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

def extract_number(filename):
    match = re.search(r'velocity_(\d+)med', filename)
    if match:
        return match.group(1)
    return None

def read_data(filepath):
    """
    Extrae datos del archivo y los devuelve como arrays de x, y, z.
    """
    timestamps = []
    values_x = []
    values_y = []
    values_z = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                # Parsear el timestamp
                timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                timestamps.append(timestamp)

                # Remover paréntesis antes de convertir a float
                coords_str = parts[1].strip().replace('(', '').replace(')', '')
                coords = list(map(float, coords_str.split(',')))  # Convertir a float
                values_x.append(coords[0])
                values_y.append(coords[1])
                values_z.append(coords[2])
    
    return np.array(values_x), np.array(values_y), np.array(values_z)

def plot_contour(x, y, z, output_file):
    """
    Genera y guarda la gráfica de curvas de nivel.
    """
    # Crear una malla de puntos (grid) para la interpolación
    grid_x, grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]

    # Interpolación de los valores de z en la malla
    grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

    # Crear la gráfica de curvas de nivel
    plt.figure(figsize=(10, 8))
    contour = plt.contourf(grid_x, grid_y, grid_z, cmap='viridis')
    plt.colorbar(contour)
    plt.title('Curvas de Nivel')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Guardar la gráfica
    plt.savefig(output_file)
    plt.close()

def process_files(file_list, output_dir):
    """
    Procesa una lista de archivos, genera y guarda gráficos de curvas de nivel para cada uno.
    """
    for file_path in file_list:
        print(file_path)
        x, y, z = read_data(file_path)
        triada = next((code for code, value in triad_codes.items() if value in os.path.basename(file_path)), None)
        if not triada:
            continue

        triada_code = triad_codes.get(triada)
        output_dir_triade = os.path.join(output_dir, triada_code)

        if not os.path.exists(output_dir_triade):
            os.makedirs(output_dir_triade)

        file_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = os.path.join(output_dir_triade, f"{file_name}_contour.png")
        plot_contour(x, y, z, output_file)

def main():
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    for shift in shifts:
        lab_dirs = []
        for lab in labs:
            lab_dir = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if lab_dir:  # Verifica si se encontró al menos un directorio
                lab_dirs.append(lab_dir[0])

        print(lab_dirs)

        if not lab_dirs:  # Verifica si no se encontraron directorios
            print("No se encontraron directorios para uno o ambos laboratorios.")
            continue

        for lab_dir in lab_dirs:
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Velocity_Experimental_Color")
            
            output_directory_baseline = os.path.join(root_folder, "Data Analysis", "Graphics", "Velocity_MEI_Nivel", lab)

            if not os.path.exists(baseline_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory_baseline):
                os.makedirs(output_directory_baseline)

            # Buscar archivos en directorios de triadas
            baseline_files = []

            for triad_code in triad_codes.values():
                baseline_files.extend(glob.glob(os.path.join(baseline_dir, triad_code, 'velocity_*med*.txt')))

            # Procesar archivos de línea base
            process_files(baseline_files, output_directory_baseline)

if __name__ == "__main__":
    main()

['F:\\Datos\\20Sep24\\20sep24 - Lab_Gamma']
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.
['F:\\Datos\\20Sep24\\20sep24 - Lab_Gamma']
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Experimental_Color\FRT\Velocity_FRT_velocity_01medamarilla.txt
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Experimental_Color\FRT\Velocity_FRT_velocity_02medroja.txt
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Experimental_Color\FRT\Velocity_FRT_velocity_03medroja.txt
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Experimental_Color\FRT\Velocity_FRT_velocity_04medroja.txt
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Processing Data\Velocity_Experimental_Color\PLB\Velocity_PLB_velocity_01medamarilla.txt
F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab

## 10

$$
\Huge \text{Comparison Moduli before and after for MEI}
$$

In [59]:
#--------------------------------------------------------------------------Triple Plot Comparación modulo rotacional-------------------------------------------------------------------------#
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import os
import glob

def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                timestamps.append(timestamp)
                values.append(float(parts[1]))
    return timestamps, np.array(values)

def get_color(filename):
    color_map = {
        'mednegra.txt': 'black',
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

def plot_triad(baseline_file, experimental_file, baseline_file2, output_dir, lab, shift, date_str):
    timestamps_baseline, baseline_values = read_data(baseline_file)
    timestamps_experimental, experimental_values = read_data(experimental_file)
    timestamps_baseline2, baseline_values2 = read_data(baseline_file2)

    # Reiniciar todas las estampas de tiempo
    start_time_baseline = timestamps_baseline[0]
    start_time_experimental = timestamps_experimental[0]
    start_time_baseline2 = timestamps_baseline2[0]
    timestamps_baseline = [(ts - start_time_baseline + datetime.datetime(1900, 1, 1)) for ts in timestamps_baseline]
    timestamps_experimental = [(ts - start_time_experimental + datetime.datetime(1900, 1, 1)) for ts in timestamps_experimental]
    timestamps_baseline2 = [(ts - start_time_baseline2 + datetime.datetime(1900, 1, 1)) for ts in timestamps_baseline2]

    fig, ax = plt.subplots(figsize=(10, 8))
    baseline_times = mdates.date2num(timestamps_baseline)
    experimental_times = mdates.date2num(timestamps_experimental)
    baseline_times2 = mdates.date2num(timestamps_baseline2)
    if lab in n_change:
        n_lab = n_change[lab]
    ax.plot(baseline_times, baseline_values, c='gray', label=os.path.basename(baseline_file))
    ax.plot(experimental_times, experimental_values, c=get_color(experimental_file), label=os.path.basename(experimental_file))
    ax.plot(baseline_times2, baseline_values2, c='black', label=os.path.basename(baseline_file2))
    ax.set_title(f'Moduli Curl double Comparison - Lab: {n_lab} - Shift: {"Vespertino" if shift == "V" else "Matutino"} - Date: {date_str}', fontsize=14)
    ax.set_xlabel('Time')
    ax.set_ylabel('Magnitude')
    ax.legend()
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.gcf().autofmt_xdate()
    plt.tight_layout()

    # Generar un nombre único para cada gráfico
    baseline_name = os.path.splitext(os.path.basename(baseline_file))[0]
    experimental_name = os.path.splitext(os.path.basename(experimental_file))[0]
    baseline_name2 = os.path.splitext(os.path.basename(baseline_file2))[0]
    output_filename = f"TRIO_{baseline_name}_VS_{experimental_name}_VS_{baseline_name2}.png"
    output_filepath = os.path.join(output_dir, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files, baseline_files2):
    combinations = []
    for base_file in baseline_files:
        base_num = int(base_file.split('med')[0][-2:])
        for exp_file in experimental_files:
            exp_num = int(exp_file.split('med')[0][-2:])
            for base_file2 in baseline_files2:
                base_num2 = int(base_file2.split('med')[0][-2:])
                if exp_num == base_num - 1 and exp_num == base_num2:
                    combinations.append((base_file, exp_file, base_file2))
    return combinations

def main():

    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    color_map = {
        "mednegra.txt": "black",
        "medroja.txt": "red",
        "medmorada.txt": "purple",
        "medazul.txt": "blue",
        "medverde.txt": "green",
        "medamarilla.txt": "yellow"
    }

    for lab in labs:
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if not lab_dirs:
                print(f"No se encontró ningún directorio para {lab}.")
                continue
            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Moduli_Curl_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Moduli_Curl_Experimental_Color")
            output_directory = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Graphics", "Moduli_Curl_Graph_Color_vs_Baseline")

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            baseline_files = [f for f in os.listdir(baseline_dir) if f.startswith('magnitude_') and f.endswith('mednegra.txt') and "00" not in f]
            experimental_files = [f for f in os.listdir(exp_dir) if f.startswith('magnitude_') and any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]
            baseline_files2 = [f for f in os.listdir(baseline_dir) if f.startswith('magnitude_') and f.endswith('mednegra.txt') and "00" not in f and f != baseline_files]

            combinations = generate_combinations(baseline_files, experimental_files, baseline_files2)

            for base_file, exp_file, base_file2 in combinations:
                baseline_file = os.path.join(baseline_dir, base_file)
                baseline_file2 = os.path.join(baseline_dir, base_file2)
                experimental_file = os.path.join(exp_dir, exp_file)
                if os.path.exists(baseline_file) and os.path.exists(experimental_file):
                    plot_triad(baseline_file, experimental_file, baseline_file2, output_directory, lab, shift, date_str)

if __name__ == "__main__":
    main()

No se encontró ningún directorio para Lab_Betta.
No se encontró ningún directorio para Lab_Betta.
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Moduli_Curl_Graph_Color_vs_Baseline\TRIO_magnitude_02mednegra_VS_magnitude_01medamarilla_VS_magnitude_01mednegra.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Moduli_Curl_Graph_Color_vs_Baseline\TRIO_magnitude_03mednegra_VS_magnitude_02medroja_VS_magnitude_02mednegra.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Moduli_Curl_Graph_Color_vs_Baseline\TRIO_magnitude_04mednegra_VS_magnitude_03medroja_VS_magnitude_03mednegra.png
Saved plot as F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Data Analysis\Graphics\Moduli_Curl_Graph_Color_vs_Baseline\TRIO_magnitude_05mednegra_VS_magnitude_04medroja_VS_magnitude_04mednegra

## 11

$$
    \Huge \text{Comparison Derivatives Histograms}
$$

In [60]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import re

def read_sensor_data(filepath):
    data = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except UnicodeDecodeError:
        print(f"Unicode decode error in file {filepath}")
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return np.array(data).T

def analyze_folder(root_folder):
    data_per_file = {}
    print(f"Analizando la carpeta raíz: {root_folder}\n")
    for root, dirs, files in os.walk(root_folder):
        dirs[:] = [d for d in dirs if d != "Data Analysis"]
        print(f"Subdirectorio: {root}")
        for filename in files:
            if filename.endswith(("negra.txt", "roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt")) and not filename.startswith("00"):
                file_path = os.path.join(root, filename)
                data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = data
                    print(f"   Encontrado y procesado: {filename}")
                else:
                    print(f"   Archivo vacío o no procesable: {filename}")
            else:
                print(f"   Archivo ignorado: {filename}")
        print("\n")
    return data_per_file

def smooth_data(data, window_length=10, polyorder=3):
    if len(data) < window_length:
        window_length = len(data) - (len(data) % 2 == 0)
    return savgol_filter(data, window_length=window_length, polyorder=polyorder)

def plot_comparison_grid(data_per_file, output_folder, title_info, lab):
    suffix_to_color = {
        "negra.txt": "black",
        "roja.txt": "red",
        "morada.txt": "purple",
        "azul.txt": "blue",
        "verde.txt": "green",
        "amarilla.txt": "yellow"
    }
    component_names = ["FRONTAL", "TRASERO", "IZQUIERDA", "DERECHA", "SUPERIOR", "INFERIOR"]

    min_length = min((len(data[0]) for data in data_per_file.values() if data.size > 0), default=None)
    if min_length is None:
        print("No se encontraron datos válidos para procesar.")
        return

    for base_key in sorted(data_per_file.keys()):
        if "negra.txt" in base_key:
            n = int(base_key.split('med')[0])
            base_data_before = data_per_file[base_key][:, :min_length]
            for exp_suffix in ["roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt"]:
                exp_key = f"{n:02d}med{exp_suffix}"
                base_key_after = f"{n+1:02d}mednegra.txt"
                
                if exp_key in data_per_file and base_key_after in data_per_file:
                    exp_data = data_per_file[exp_key][:, :min_length]
                    base_data_after = data_per_file[base_key_after][:, :min_length]

                    fig, axes = plt.subplots(6, 2, figsize=(40, 60), dpi=300)
                    for i, component in enumerate(component_names):
                        ax_left = axes[i, 0]
                        ax_right = axes[i, 1]
                        smoothed_base_before = smooth_data(base_data_before[i])
                        smoothed_exp = smooth_data(exp_data[i])
                        smoothed_base_after = smooth_data(base_data_after[i])
                        derivative_base_before = np.gradient(smoothed_base_before)
                        derivative_exp = np.gradient(smoothed_exp)
                        derivative_base_after = np.gradient(smoothed_base_after)

                        # Histograma ajustado a formato similar a Mathematica
                        ax_left.hist(derivative_base_before, bins=25, color='black', alpha=0.5, label=f'Base Antes: {base_key}', edgecolor='black', density=True, log=True)
                        ax_left.hist(derivative_exp, bins=25, color=suffix_to_color[exp_suffix], alpha=0.7, label=f'Experimental: {exp_key}', edgecolor='black', density=True, log=True)
                        ax_left.set_title(f"{component} - Antes vs Exp", fontsize=30)
                        ax_left.set_xlabel("Valor Derivado", fontsize=25)
                        ax_left.set_ylabel("Probabilidad", fontsize=25)
                        ax_left.legend(fontsize=25)
                        ax_left.tick_params(axis='both', labelsize=35)

                        ax_right.hist(derivative_base_after, bins=25, color='black', alpha=0.5, label=f'Base Después: {base_key_after}', edgecolor='black', density=True, log=True)
                        ax_right.hist(derivative_exp, bins=25, color=suffix_to_color[exp_suffix], alpha=0.7, label=f'Experimental: {exp_key}', edgecolor='black', density=True, log=True)
                        ax_right.set_title(f"{component} - Después vs Exp", fontsize=30)
                        ax_right.set_xlabel("Valor Derivado", fontsize=25)
                        ax_right.set_ylabel("Probabilidad", fontsize=25)
                        ax_right.legend(fontsize=25)
                        ax_right.tick_params(axis='both', labelsize=35)

                    date, shift = title_info
                    shift_full = "MATUTINO" if shift == "M" else "VESPERTINO"
                    plt.suptitle(f"Comparaciones de Derivadas - {date} - {shift_full} - {lab}", fontsize=50)
                    plt.tight_layout(pad=5.0)
                    output_file = os.path.join(output_folder, f"deriv_{n:02d}_{exp_suffix.split('.')[0]}.pdf")
                    plt.savefig(output_file, bbox_inches='tight', format='pdf')
                    plt.close()

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def main():
    laboratorios = ["Lab_Betta", "Lab_Gamma"]
    turnos = ["M", "V"]

    for lab in laboratorios:
        for turno in turnos:
            experiment_folder, date_str = find_experiment_folder(root_folder, lab, turno)
            if not experiment_folder:
                print(f"No se encontró ningún directorio para {lab}, {turno}.")
                continue

            output_folder = os.path.join(experiment_folder, "Data Analysis", "Graphics", "Comparison_D[R]")
            os.makedirs(output_folder, exist_ok=True)

            title_info = (date_str, turno)

            data_per_file = analyze_folder(experiment_folder)
            plot_comparison_grid(data_per_file, output_folder, title_info, lab)

if __name__ == "__main__":
    main()


No se encontró ningún directorio para Lab_Betta, M.
No se encontró ningún directorio para Lab_Betta, V.
Analizando la carpeta raíz: F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma

Subdirectorio: F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma
   Archivo ignorado: Bitacora 20-09-24.txt


Subdirectorio: F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Amarilla
   Encontrado y procesado: 01medamarilla.txt


Subdirectorio: F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Negra
   Archivo ignorado: 001mednegra.txt
   Archivo ignorado: 002mednegra.txt
   Encontrado y procesado: 01mednegra.txt
   Encontrado y procesado: 02mednegra.txt
   Encontrado y procesado: 03mednegra.txt
   Encontrado y procesado: 04mednegra.txt
   Encontrado y procesado: 05mednegra.txt


Subdirectorio: F:\Datos\20Sep24\20sep24 - Lab_Gamma\20sep24.M - Lab_Gamma\Pesajes
   Archivo ignorado: 01medamarilla.log
   Archivo ignorado: 01mednegra.log
   Archivo ignorado: 02mednegra.log



## 12

$$
    \Huge \text{Comparison Time Series Laser}
$$



In [61]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import re

def read_sensor_data(filepath):
    data = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except UnicodeDecodeError:
        print(f"Unicode decode error in file {filepath}")
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return np.array(data).T

def analyze_folder(root_folder):
    data_per_file = {}
    for root, dirs, files in os.walk(root_folder):
        dirs[:] = [d for d in dirs if d != "Data Analysis"]
        for filename in files:
            if filename.endswith(("negra.txt", "roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt", "naranja.txt")) and not filename.startswith("00"):
                file_path = os.path.join(root, filename)
                data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = data
    return data_per_file

def smooth_data(data, window_length=10, polyorder=3):
    if len(data) < window_length:
        window_length = len(data) - (len(data) % 2 == 0)
    return savgol_filter(data, window_length=window_length, polyorder=polyorder)

def plot_comparison_grid(data_per_file, output_folder, title_info, lab):
    suffix_to_color = {
        "negra.txt": "black",
        "roja.txt": "red",
        "morada.txt": "purple",
        "azul.txt": "blue",
        "verde.txt": "green",
        "amarilla.txt": "yellow",
        "naranja.txt": "orange"
    }
    component_names = ["FRONTAL", "TRASERO", "IZQUIERDA", "DERECHA", "SUPERIOR", "INFERIOR"]

    for base_key in sorted(data_per_file.keys()):
        if "negra.txt" in base_key:
            try:
                n = int(re.search(r'\d+', base_key.split('med')[0]).group())
            except (ValueError, AttributeError):
                print(f"El archivo {base_key} no contiene un número válido para conversión.")
                continue

            base_data_before = data_per_file[base_key]
            for exp_suffix in ["roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt", "naranja.txt"]:
                exp_key = f"{n:02d}med{exp_suffix}"
                base_key_after = f"{n+1:02d}mednegra.txt"
                if exp_key in data_per_file and base_key_after in data_per_file:
                    exp_data = data_per_file[exp_key]
                    base_data_after = data_per_file[base_key_after]
                    fig, axes = plt.subplots(6, 2, figsize=(40, 60), dpi=300)
                    axes = axes.reshape(-1, 2)
                    for i, component in enumerate(component_names):
                        ax_before = axes[i, 0]
                        ax_after = axes[i, 1]
                        smoothed_base_before = smooth_data(base_data_before[i])
                        smoothed_exp = smooth_data(exp_data[i])
                        smoothed_base_after = smooth_data(base_data_after[i])
                        ax_before.plot(smoothed_base_before, label=f'Línea Base: {base_key}', color='black', linewidth=2)
                        ax_before.plot(smoothed_exp, label=f'Experimental: {exp_key}', color=suffix_to_color[exp_suffix], linewidth=2)
                        ax_before.set_title(f"Componente {component} (Antes)", fontsize=25)
                        ax_before.set_xlabel("Registros", fontsize=25)
                        ax_before.set_ylabel("Volts", fontsize=25)
                        ax_before.tick_params(axis='both', which='major', labelsize=20)
                        ax_before.legend(fontsize=30)
                        ax_after.plot(smoothed_base_after, label=f'Línea Base: {base_key_after}', color='black', linewidth=2)
                        ax_after.plot(smoothed_exp, label=f'Experimental: {exp_key}', color=suffix_to_color[exp_suffix], linewidth=2)
                        ax_after.set_title(f"Componente {component} (Después)", fontsize=25)
                        ax_after.set_xlabel("Registros", fontsize=25)
                        ax_after.set_ylabel("Volts", fontsize=25)
                        ax_after.tick_params(axis='both', which='major', labelsize=20)
                        ax_after.legend(fontsize=30)
                    date, shift = title_info
                    shift_full = "MATUTINO" if shift == "M" else "VESPERTINO"
                    color_name = exp_suffix.split('.')[0].upper()
                    output_file = os.path.join(output_folder, f"comparacion_{n:02d}_{n+1:02d}_base_{n}_{exp_suffix.split('.')[0]}.pdf")
                    plt.suptitle(f"COMPARACIÓN BASE {n:02d} Y DESPUÉS {n+1:02d} DE INTERVENCIÓN - {date} - {shift_full} - INTERVENCIÓN: {color_name} - LABORATORIO: {lab}", fontsize=35)
                    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                    plt.savefig(output_file, bbox_inches='tight', format='pdf')
                    plt.close()

def find_experiment_folder(root_folder, lab, turno):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and turno in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def main():
    # Definir los laboratorios y turnos
    laboratorios = ["Lab_Betta", "Lab_Gamma"]
    turnos = ["M", "V"]


    for lab in laboratorios:
        for turno in turnos:
            # Encontrar la carpeta del experimento correspondiente al laboratorio y turno
            experiment_folder, date_str = find_experiment_folder(root_folder, lab, turno)
            if not experiment_folder:
                print(f"No se encontró ningún directorio para {lab}, {turno}.")
                continue

            # Acceder directamente a la carpeta del experimento para procesar datos sin entrar a "Data Analysis"
            output_folder = os.path.join(experiment_folder,"Data Analysis", "Graphics", "Comparison")
            os.makedirs(output_folder, exist_ok=True)  # Crear la carpeta de gráficos si no existe

            title_info = (date_str, turno)

            # Analizar los archivos en la carpeta del experimento directamente
            data_per_file = analyze_folder(experiment_folder)
            # Generar las gráficas y guardarlas en la carpeta de gráficos
            plot_comparison_grid(data_per_file, output_folder, title_info, lab)

if __name__ == "__main__":
    main()


No se encontró ningún directorio para Lab_Betta, M.
No se encontró ningún directorio para Lab_Betta, V.
No se encontró ningún directorio para Lab_Gamma, V.


In [62]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
import glob

# Función para extraer la fecha del nombre del directorio raíz
def extraer_fecha(root_folder):
    nombre_directorio = os.path.basename(root_folder)
    match = re.search(r'(\d{2}[A-Za-z]{3}\d{2})', nombre_directorio)
    
    if match:
        fecha_str = match.group(1)  # Captura la fecha en formato '16Sep24'
        return fecha_str
    else:
        raise ValueError(f"No se encontró una fecha válida en el nombre del directorio {root_folder}.")

# Función para cargar datos desde un archivo .txt
def cargar_datos_txt(ruta_archivo):
    datos = pd.read_csv(ruta_archivo, header=None, names=['Hora', 'Valor'])
    datos['Hora'] = pd.to_timedelta(datos['Hora'])  # Convertir la columna 'Hora' a formato de tiempo
    datos['Hora_Segundos'] = datos['Hora'].dt.total_seconds()  # Convertir Hora a segundos
    return datos

# Función para graficar comparando las mediciones negras y de colores
def graficar_comparacion_txt(ruta_archivo_negra, ruta_archivo_color, color_grafico, ruta_salida, titulo='Comparación de Datos'):
    # Cargar datos de ambas mediciones (negra y de color)
    datos_negra = cargar_datos_txt(ruta_archivo_negra)
    datos_color = cargar_datos_txt(ruta_archivo_color)
    
    # Crear la figura y graficar ambas mediciones
    plt.figure(figsize=(10, 5))
    
    # Graficar datos de la medición negra (siempre en negro)
    plt.scatter(datos_negra['Hora_Segundos'], datos_negra['Valor'], color='black', label='Negra ⚫')
    
    # Graficar datos de la medición de color (usar el color según el archivo)
    plt.scatter(datos_color['Hora_Segundos'], datos_color['Valor'], color=color_grafico, label=f'Medición {color_grafico.capitalize()}')
    
    # Añadir título y etiquetas
    plt.title(titulo)
    plt.xlabel('Hora (segundos)')
    plt.ylabel('Valor (g)')
    plt.grid()
    plt.legend()
    plt.tight_layout()
    
    # Guardar la gráfica
    plt.savefig(ruta_salida)
    plt.close()  # Cerrar la figura para liberar memoria

# Función principal para graficar comparando las mediciones negras y de colores
def procesar_graficos(root_folder):
    labs = ["Betta", "Gamma"]
    turnos = ["M", "V"]
    colores = ["Amarilla", "Roja", "Azul", "Verde", "Morada"]

    fecha = extraer_fecha(root_folder)

    # Diccionario para asignar colores a los puntos de las gráficas
    color_map = {
        "amarilla": "yellow",
        "roja": "red",
        "azul": "blue",
        "verde": "green",
        "morada": "purple"
    }

    for lab in labs:
        for turno in turnos:
            for color in colores:
                colorin = color.lower()
                
                # Ruta del directorio donde están los archivos procesados
                carpeta_procesados = os.path.join(
                    root_folder, 
                    f"{fecha} - Lab_{lab}", 
                    f"{fecha}.{turno} - Lab_{lab}", 
                    "Pesajes", 
                    "procesados"
                )
                
                # Buscar todos los archivos negros
                archivos_negros = glob.glob(os.path.join(carpeta_procesados, "resultados_*_negra.txt"))

                # Buscar todos los archivos de color actual
                archivos_color = glob.glob(os.path.join(carpeta_procesados, f"resultados_*_{colorin}.txt"))
                
                if not archivos_negros or not archivos_color:
                    print(f"No se encontraron archivos para la comparación en Lab_{lab} Turno_{turno} Color_{color}.")
                    continue

                # Crear carpeta de salida para los gráficos
                output_folder = os.path.join(root_folder, f"{fecha} - Lab_{lab}", "Data Analysis", "Graphics", "Pesajes_Comparados")
                os.makedirs(output_folder, exist_ok=True)

                # Comparar todos los archivos negros con los archivos de color (independiente del número)
                for archivo_negra in archivos_negros:
                    for archivo_color in archivos_color:
                        # Generar el nombre del archivo de salida
                        numero_negra = re.search(r'resultados_(\d+)_negra', archivo_negra).group(1)
                        numero_color = re.search(r'resultados_(\d+)_' + colorin, archivo_color).group(1)
                        
                        # Ruta de salida
                        ruta_salida = os.path.join(output_folder, f"grafica_comparada_{lab}_{turno}_{color}_{numero_negra}_{numero_color}.png")
                        
                        # Obtener el color de la gráfica según el archivo de color
                        color_grafico = color_map.get(colorin, 'blue')  # Usa azul como predeterminado si no está en el mapa

                        # Graficar la comparación
                        graficar_comparacion_txt(archivo_negra, archivo_color, color_grafico, ruta_salida, 
                                                 titulo=f'Comparación {lab} {turno} {color} Negra-{numero_negra} Color-{numero_color}')
                        print(f"Gráfica comparativa guardada en: {ruta_salida}")

# Si este es el archivo principal
if __name__ == "__main__":
    procesar_graficos(root_folder)
    print("\nGraficación comparativa de archivos procesados completa.")

No se encontraron archivos para la comparación en Lab_Betta Turno_M Color_Amarilla.
No se encontraron archivos para la comparación en Lab_Betta Turno_M Color_Roja.
No se encontraron archivos para la comparación en Lab_Betta Turno_M Color_Azul.
No se encontraron archivos para la comparación en Lab_Betta Turno_M Color_Verde.
No se encontraron archivos para la comparación en Lab_Betta Turno_M Color_Morada.
No se encontraron archivos para la comparación en Lab_Betta Turno_V Color_Amarilla.
No se encontraron archivos para la comparación en Lab_Betta Turno_V Color_Roja.
No se encontraron archivos para la comparación en Lab_Betta Turno_V Color_Azul.
No se encontraron archivos para la comparación en Lab_Betta Turno_V Color_Verde.
No se encontraron archivos para la comparación en Lab_Betta Turno_V Color_Morada.
Gráfica comparativa guardada en: F:\Datos\20Sep24\20Sep24 - Lab_Gamma\Data Analysis\Graphics\Pesajes_Comparados\grafica_comparada_Gamma_M_Amarilla_01_01.png
Gráfica comparativa guardada 